# Clasificación de Tumores Cerebrales con CNN (Entrenamiento y Validación 80/20)

Este notebook está preparado para el desarrollo y entrenamiento de una CNN sobre imágenes MRI. Incluye:
- Separación automática 80/20 para entrenamiento y validación
- Visualización de datos
- Definición y entrenamiento del modelo
- Guardado automático de checkpoints cada N épocas
- Tamaño de imagen: 512x512 píxeles

In [ ]:
# Importar bibliotecas necesarias
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

# Configuración de semilla para reproducibilidad
np.random.seed(42)
tf.random.set_seed(42)

# Comprobar si hay GPU disponible
print("Dispositivos disponibles:")
for device in tf.config.list_physical_devices():
    print(f"  {device.name}")
print(f"¿GPU disponible? {'GPU' in [d.device_type for d in tf.config.list_physical_devices()]}")

In [ ]:
# Configuración de rutas
BASE_DIR = Path(os.path.join(os.getcwd(), '../total/archive/Training'))
print(f"BASE_DIR: {BASE_DIR}")

# Detectar clases automáticamente
classes = [d.name for d in BASE_DIR.iterdir() if d.is_dir()]
print(f"Clases detectadas: {classes}")
num_classes = len(classes)

# Parámetros
IMG_SIZE = 512  # Tamaño de imagen actualizado a 512x512
BATCH_SIZE = 32
EPOCHS = 200
LEARNING_RATE = 0.001
CHECKPOINT_EVERY = 5  # Guardar cada N épocas
MODEL_DIR = Path(os.path.join(os.getcwd(), '../models'))
MODEL_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
# Generador de datos con separación 80/20 (entrenamiento/validación)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=False,
    fill_mode='nearest',
    validation_split=0.2  # 80% train, 20% val
)

train_generator = train_datagen.flow_from_directory(
    str(BASE_DIR),
    target_size=(IMG_SIZE, IMG_SIZE),  # Actualizado a 512x512
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_generator = train_datagen.flow_from_directory(
    str(BASE_DIR),
    target_size=(IMG_SIZE, IMG_SIZE),  # Actualizado a 512x512
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

class_indices = train_generator.class_indices
class_names = {v: k for k, v in class_indices.items()}
print("Mapeo de clases:", class_indices)

In [ ]:
# Visualizar algunas imágenes de entrenamiento
plt.figure(figsize=(15, 8))
images, labels = next(train_generator)
labels = np.argmax(labels, axis=1)
for i in range(min(10, len(images))):
    plt.subplot(2, 5, i + 1)
    plt.imshow(images[i])
    plt.title(f"Clase: {class_names[labels[i]]}")
    plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Definir arquitectura de la CNN
def create_model(input_shape=(IMG_SIZE, IMG_SIZE, 3), num_classes=num_classes):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

model = create_model()
model.summary()

In [ ]:
# Callbacks: EarlyStopping, ReduceLROnPlateau, y ModelCheckpoint cada N épocas
checkpoint_path = str(MODEL_DIR / 'cnn_checkpoint_epoch_{epoch:02d}.h5')
model_checkpoint = ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=False,
    save_freq='epoch',
    period=CHECKPOINT_EVERY,  # Guardar cada N épocas
    verbose=1
)
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6, verbose=1)
callbacks = [model_checkpoint, early_stop, reduce_lr]

In [ ]:
# Entrenamiento del modelo con validación
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=val_generator,
    validation_steps=val_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=1
)

In [ ]:
# Guardar el modelo final
final_model_path = MODEL_DIR / 'brain_tumor_cnn_model_final.h5'
model.save(str(final_model_path))
print(f"Modelo final guardado en: {final_model_path}")

In [ ]:
# Visualización de la curva de pérdida y precisión
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Precisión del modelo')
plt.ylabel('Precisión')
plt.xlabel('Época')
plt.legend(['Entrenamiento', 'Validación'], loc='lower right')
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Pérdida del modelo')
plt.ylabel('Pérdida')
plt.xlabel('Época')
plt.legend(['Entrenamiento', 'Validación'], loc='upper right')
plt.tight_layout()
plt.show()

**Notas:**
- El entrenamiento se realiza con separación 80/20 para entrenamiento y validación.
- Se guardan checkpoints automáticos cada N épocas y el modelo final al terminar.
- Puedes ajustar los hiperparámetros y arquitectura según tus necesidades.